# Regularizing a deep neural network

## Setup

In [2]:
import os
if os.getcwd().endswith("notebooks"):
    os.chdir('..')

print("Current working directory: ", os.getcwd())
if not os.getcwd().endswith("CIFAR-100-DNN-Optimization"):
    raise ValueError("Please change working directory to 'path/CIFAR-100-DNN-Optimization' before proceeding")

Current working directory:  /Users/irellzane/MLprojects/CIFAR-100-DNN-Optimization


In [3]:
!pip install -r requirements.txt

In [4]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [17]:
from tensorflow.keras.layers import Dense, Flatten, Input, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [6]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        self.model.optimizer.learning_rate = rate

## Load Data

In [7]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

data_pre_path = './data/CIFAR-100/'

In [8]:
data_train_path = data_pre_path + 'train'
data_test_path = data_pre_path + 'test'
data_train_dict = unpickle(data_train_path)
data_test_dict = unpickle(data_test_path)
data_train = data_train_dict[b'data']
y_train_valid = np.array(data_train_dict[b'fine_labels'])
data_test = data_test_dict[b'data']
y_test = np.array(data_test_dict[b'fine_labels'])

In [9]:
X_train_valid = data_train.reshape(-1, 3, 32, 32).transpose(0,2,3,1) / 255
X_test = data_test.reshape(-1, 3, 32, 32).transpose(0,2,3,1) / 255

In [14]:
input_shape = X_train_valid[0].shape
input_shape

(32, 32, 3)

## Data splitting

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=5000/50000, stratify=y_train_valid, random_state=42)

## L2 regularization

In [21]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Flatten())

for _ in range(10):
    model.add(Dense(400, activation="swish", kernel_initializer="he_normal", kernel_regularizer=l2(0.01)))
    
model.add(Dense(100, activation="softmax"))

optimizer = tf.keras.optimizers.SGD(nesterov=True, learning_rate=0.05)

model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [22]:
n_epochs = 30
batch_size = 128
onecycle = OneCycleScheduler(np.ceil(len(X_train) / batch_size) * n_epochs,
                             max_rate=0.5)

In [23]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=n_epochs,
    batch_size=batch_size, 
    callbacks=[onecycle]
)

Epoch 1/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.0130 - loss: 71.0393 - val_accuracy: 0.0200 - val_loss: 35.8420
Epoch 2/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.0217 - loss: 28.2146 - val_accuracy: 0.0196 - val_loss: 12.2408
Epoch 3/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.0190 - loss: 9.9001 - val_accuracy: 0.0194 - val_loss: 5.7887
Epoch 4/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.0137 - loss: 5.3799 - val_accuracy: 0.0100 - val_loss: 4.7293
Epoch 5/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.0090 - loss: 4.6799 - val_accuracy: 0.0100 - val_loss: 4.6127
Epoch 6/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.0092 - loss: 4.6105 - val_accuracy: 0.0100 - val_loss: 4.6056
Epoch 7/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.0087 - loss: 4.6066 - val_accuracy: 0.0100 - val_loss: 4.6053
Epoch 8/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.0091 - loss: 4.606

In [24]:
plot_history(history)

NameError: name 'plot_history' is not defined

In [ ]:
model.evaluate(X_valid, y_valid)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2046 - loss: 6.4317


[6.342916488647461, 0.21040000021457672]

In [ ]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2079 - loss: 6.4189


[6.3947062492370605, 0.20890000462532043]